## __packages__

In [1]:
import pandas as pd
import mysql.connector as msql
from mysql.connector import Error
from mysql_config import config_read

## __class__

In [2]:

class Table:
    
    def __init__(self, name, table):
        self.name = name
        self.table = table
        self.insert_query = self.make_insert_query()
        
    def make_insert_query(self):
        values_tuple = ",".join(["%s" for i in range(self.table.shape[1])])
        insert_query = "INSERT INTO spotify.%s VALUES (%s)" % (self.name, values_tuple)
        
        return insert_query
                

## __load tables__

In [3]:
table_path = "./sample_%s.csv"
table_list = ["artist", "album", "track", "audio_features", "artist_album", "artist_track"]

for table in table_list:
    
    tmp = pd.read_csv(table_path % table)
    
    globals()[table] = Table(table, tmp.copy())


## __connect with the server and import data__

In [4]:
try:
    conn = msql.connect(
        host="localhost", 
        user="root", 
        password=config_read("authentication", "password"), 
        database="spotify", 
        auth_plugin="mysql_native_password"
    )
    
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("USE spotify;")
        
        for table in table_list:
            
            tmp_object = globals()[table]
            
            for i, row in tmp_object.table.iterrows():
                
                sql = tmp_object.insert_query
                
                cursor.execute(sql, tuple(row))
                
                conn.commit()
            


except Error as e:
    print("Error while connecting to MySQL", e)

finally:
    conn.close()